In [98]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor

In [2]:
# ucitvanje podataka

data = pd.read_csv('./Google-Playstore.csv')
population = pd.read_csv('./Countries-Population.csv') # podaci iz 2018. godine

In [3]:
# odbacivanje nepotrebnih podataka

data.drop('App Name', axis=1, inplace=True)
data.drop('App Id', axis=1, inplace=True)
data.drop('Scraped Time', axis=1, inplace=True)
data.drop('Developer Website', axis=1, inplace=True)
data.drop('Developer Email', axis=1, inplace=True)
data.drop('Privacy Policy', axis=1, inplace=True)
data.drop('Currency', axis=1, inplace=True)
data.drop('Installs', axis=1, inplace=True)
data.drop('Minimum Installs', axis=1, inplace=True)

In [4]:
# Kodiranje

data['Free'].replace({True: 1, False: 0}, inplace=True)
data['Ad Supported'].replace({True: 1, False: 0}, inplace=True)
data['In App Purchases'].replace({True: 1, False: 0}, inplace=True)
data['Editors Choice'].replace({True: 1, False: 0}, inplace=True)

/var/folders/wc/x85w4mlj5m12w_fs1ctclp6h0000gn/T/ipykernel_21294/670095493.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Free'].replace({True: 1, False: 0}, inplace=True)
/var/folders/wc/x85w4mlj5m12w_fs1ctclp6h0000gn/T/ipykernel_21294/670095493.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting',

In [5]:
# prikupljanje svih imena iz kolone "Category" i pravljenje novih kolona

list_of_categories = data['Category'].unique()
categories = pd.get_dummies(data['Category'])
categories.replace({True: 1, False: 0}, inplace=True)
data = pd.concat([data, categories], axis=1)
data.drop('Category', axis=1, inplace=True)

_ = pd.get_dummies(data['Content Rating'])
_.replace({True: 1, False: 0}, inplace=True)
data = pd.concat([data, _], axis=1)
data.drop('Content Rating', axis=1, inplace=True)

/var/folders/wc/x85w4mlj5m12w_fs1ctclp6h0000gn/T/ipykernel_21294/2446235330.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  categories.replace({True: 1, False: 0}, inplace=True)
/var/folders/wc/x85w4mlj5m12w_fs1ctclp6h0000gn/T/ipykernel_21294/2446235330.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  _.replace({True: 1, False: 0}, inplace=True)


In [6]:
# generisanje jedinstvenog ID za svakog developera

_ = pd.DataFrame()
_['Developer'] = data['Developer Id'].unique()
_['Id'] = range(1, len(_) + 1)

developers_dict = dict(zip(_['Developer'], _['Id']))

data['Developer Id'] = data['Developer Id'].map(developers_dict)

In [7]:
# konvertovanje velicine aplikacije

def convert_to_mb(size):
    size_str = str(size)

    if (size_str[-1] == 'k'):
        return float(size_str[:-1].replace(',', '.')) / 1024

    if (size_str[-1] == 'M'):
        return float(size_str[:-1].replace(',', '.'))

    if (size_str[-1] == 'G'):
        return float(size_str[:-1]) * 1024.0

    else:
        return None

data['Size'] = data['Size'].apply(lambda x: convert_to_mb(x))

In [8]:
# prevodjenje formata datuma

data['Released'] = pd.to_datetime(data['Released'])
data['Released'] = data['Released'].dt.strftime('%d.%m.%Y') 
data['Last Updated'] = pd.to_datetime(data['Last Updated'])
data['Last Updated'] = data['Last Updated'].dt.strftime('%d.%m.%Y') 

In [9]:
# konvertovanje android verzija u datume

android_versions_dict = pd.read_csv('./Android-Versions.csv')
android_versions_dict['Android version'] = android_versions_dict['Android version'].apply(lambda x: str(x))

android_versions_release_date = pd.DataFrame(android_versions_dict['Release date'].copy(deep=True))

android_versions_dict = dict(zip(android_versions_dict['Android version'], android_versions_dict['Release date']))

def iseci(x):
    return str(x)[:3]

data['Minimum Android'] = data['Minimum Android'].apply(lambda x: iseci(x)) 

# Posto nemamo podatke o subverzijama androida kada su izasle, mora da gledamo samo verzije.
android_versions_dict.update({'nan': 0})
android_versions_dict.update({'Var': 'Varies with device'})

data['Minimum Android'] = data['Minimum Android'].map(android_versions_dict)

In [10]:
# dopunjavanje NaN vrednosti
# Pretpostavljamo da aplikacija nije imala rating-ove

data['Rating'].fillna(0.0, inplace=True)
data['Rating Count'].fillna(0.0, inplace=True)

/var/folders/wc/x85w4mlj5m12w_fs1ctclp6h0000gn/T/ipykernel_21294/784949730.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Rating'].fillna(0.0, inplace=True)
/var/folders/wc/x85w4mlj5m12w_fs1ctclp6h0000gn/T/ipykernel_21294/784949730.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves

In [11]:
for category in list_of_categories:

    average_size = data.loc[data[category] == 1]['Size'].mean()
    
    #print(category)
    #print(average_size)
    #plt.plot(data.loc[data[category] == 1]['Size'])
    #plt.axhline(average_size, color='r', linestyle='--', label='Mean')
    #plt.show()

    _ = pd.DataFrame(data.loc[data['Size'].isna()][category])
    indexes = _.index[_[category] == 1].to_list()
    data.loc[indexes, 'Size'] = average_size


In [12]:
# izdvajanje svih indeksa u koloni 'Minimum Android' gde je vrednost polja jednaka 'Varies with device' 
# zato sto ce missing podaci u koloni 'Released' biti popunjeni vrednoscu one verzije koja je minimalna da bi ta aplikacija radila

# verovatno da je aplikacija objavljena pre toga ali ovo je najblize i najpreciznije sto mozemo pretpostaviti.

to_be_removed = data.loc[data['Minimum Android'] == 'Varies with device'].index.to_list()
data.drop(to_be_removed, inplace=True)

to_be_fixed = data.loc[data['Released'].isna()].index.to_list()
data.loc[to_be_fixed, 'Released'] = data.loc[to_be_fixed]['Minimum Android']

In [80]:
# odbacivanje podataka kojima ne znamo ni minimalnu potrebnu verziju Android-a niti kada je objavljena

to_be_removed = data.loc[data['Released'].isna()].index.to_list()
data.drop(to_be_removed, inplace=True)
to_be_removed = data.loc[data['Released'] == 0].index.to_list()
data.drop(to_be_removed, inplace=True)

In [81]:
# dopunjavanje NaN vrednosti u koloni Minimum Android
# dopunjavanje se vrsi tako sto se nadje prva verzija android-ida koja je izasla pre aplikacije

def days_since_epoch(date_string): # vraca broj dana od 1.1.1970 do prosledjenog datuma
    date_object = datetime.strptime(date_string, "%d.%m.%Y")
    epoch = datetime(1970, 1, 1)
    delta = date_object - epoch
    return delta.days

android_versions_release_date['since_111970'] = android_versions_release_date['Release date'].apply(lambda x: days_since_epoch(x))

def nadji_verziju(x):
    searched = days_since_epoch(x)
    for index in range(android_versions_release_date['since_111970'].count()):
        if searched <= android_versions_release_date['since_111970'][index]:
            if index == 0:
                return 'to_be_droped'
            return android_versions_release_date['Release date'][index-1]
        
to_be_fixed = data.loc[data['Minimum Android'].isna()].index.to_list()
data.loc[to_be_fixed, 'Minimum Android'] = data.loc[to_be_fixed, 'Released'].apply(lambda x: nadji_verziju(x))

In [85]:
# neke problematicne vrednosti
 
to_be_removed = data.loc[data['Minimum Android'] == 0].index.to_list()
data.drop(to_be_removed, inplace=True)

In [86]:
# Datum mora da se prevede u broj

data['Minimum Android'] = data['Minimum Android'].apply(lambda x: days_since_epoch(x))
data['Released'] = data['Released'].apply(lambda x: days_since_epoch(x))
data['Last Updated'] = data['Last Updated'].apply(lambda x: days_since_epoch(x))

In [88]:
plt.figure(figsize=(200,100))
sns.heatmap(data.corr(), annot=True, cmap='YlGnBu')

<Axes: >

In [100]:
# pravljenje modela

y = data['Maximum Installs'].copy(deep=True)
X = data.copy(deep=True)
X.drop('Maximum Installs', axis=1)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

KeyboardInterrupt: 